In [1]:
import numpy as np
from scipy.optimize import linprog, minimize_scalar
from numdifftools import Gradient

In [2]:
def f(x):
    return 0.5 * ((x[0] ** 2) + (x[1] ** 2))

A = np.array([[-1, 1], [1, 1], [0, -1]], dtype=float)
b = [7, 5, -2]

# Feasible directions algorithm

In [3]:
def feasible_directions(f, A, b, x0, tol):
    x = x0
    m = len(A)

    while True:
        c = Gradient(f)(x)
        I = []
        for i in range(m):
            if abs(np.vdot(A[i], x) - b[i]) < tol:
                I.append(i)
        A_prog = A[I]
        b_prog = np.zeros(len(I))
        d = linprog(c, A_ub=A_prog, b_ub=b_prog, bounds=(-1, 1)).x

        alpha_bar = np.inf
        for i in range(m):
            a = np.vdot(A[i], d)
            if (i not in I) and (a > 0):
                alpha = (b[i] - np.vdot(A[i], x)) / a
                alpha_bar = min(alpha_bar, alpha)

        phi = lambda alpha: f(x + alpha * d)
        alpha = minimize_scalar(phi, bounds=(0, alpha_bar), method='bounded').x
        x = x + alpha * d

        if np.vdot(Gradient(f)(x), d) >= 0:
            return x

In [4]:
x0 = [-2, 3]
x_sol = feasible_directions(f, A, b, x0, 0.0001)
print(x_sol)

[0.         2.00000596]
